<img style="float: left;" src="images/unige_csd.png" alt="drawing" width="200"/>

# <center>Imagerie Numérique 2023 Automne</center>

<center>November 10, 2023</center>

# <center><ins>TP Class N°4 - Histograms and point operations</ins></center>

#### *Instructions :*  

* This TP should be completed and uploaded on Moodle before **Thursday 23 November 2023, 23h59**.
* The name of the file you upload should be **TP4_name_surname.ipynb**.
* If you need to include attached files to you TP, please archive them together in a folder named **TP4_name_surname.zip**.

## Exercise 1. Exposure analysis

You are given 3 images: _img\_1.png_, _img\_2.png_ and _img\_3.png_.

(a) Plot the histograms of these images using the _histogram(.)_ function from the _skimage_ package.

(b) Try to detect if these images have problems with exposure. If yes, identify the reasons of these problems via histogram analysis.

> Answer here

(c) Suggest an automatic procedure for detection of under/over-exposure in images.

> Answer here

## Exercise 2. Contrast analysis

You are given 3 images: _img\_4.png_, _img\_5.png_ and _img\_6.png_.

Detect and identify contrast problems in these images, if any. Justify your answer based on the analysis of the histograms.

## Exercise 3. Contrast adjustment

**Important!** For color images, the histogram analysis should be performed individually on each color channel.

(a) Write a function that performs an image contrast adjustment (stretching). For more details see Theme 4.

(b) Apply your contrast adjustment function to the image _img 9.png_. Show image histogram before and after adjustment. Explain the result.

(c) Apply the _rescale intensity(.)_ function from the _skimage_ package to the original image. Compare the result with the results of your function. Explain the differences in image quality, if any.

(d) Perform the sub-exercise (c) with _in_range_ parameter defined by using _np.percentile(.)_ with _q_ parameter equals to (5, 95). Explain the difference between (c) and (d).

> **Hint** : <br/>
start, end = np.percentile(image, (5, 95)) <br/>
adjusted image = rescale intensity(image, in range=(start, end))

## Exercise 4. Histogram equalization

(a) Perform histogram equalization to the image _img\_11.png_ by using _equalize\_hist(.)_ function from the _skimage_ package.

(b) Display the image histogram before and after equalization. Explain what the histogram equalization does.

## Exercise 5. Histogram matching

(a) Apply the function _match_histograms(.)_ from _skimage_ package to _img_4.png_ using _img_2.png_ as a reference.

(b) Display the histograms of _img_2.png_, _img_4.png_ before transformation and _img_4.png_ after transformation.

(c) Answer to the following questions:

• What does this function do ? <br/>
• How does this function work ? <br/>
• What can we use it for ?

> Answer here

## Exercise 6. Gamma Correction

(a) Write a function that performs image gamma correction. For more details see Theme 4.

(b) Apply your function to img 3.png, for various gamma values. Try to determine the gamma value that best improves the visual image quality.

(c) Visualize the image before and after correction. Explain the result.

## Exercise 7. Digital negatives

(a) Write a function that transforms an image into its digital negative.

(b) Choose 2 images from _img_1.png_ to _img_8.png_ and compute their digital negative. Visualize both the original and negative images.

(c) Explain in which situation this transformation could be useful.

> Answer here

## Exercise 8. Image masking

You are given the image _img_10.png_.

(a) Perform image thresholding to obtain a binary segmentation mask. Display the results for various choices of threshold values.

(b) Perform a pixel-wise multiplication between the original image and the various segmentation masks to obtain masked versions of the original image at different threshold. Display the results.

(c) Which threshold value works best for the segmentation of the statuette ?

> Answer here